In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../datascience/data/pbp-2024.csv')

Source Data From: [NFL Savant](https://nflsavant.com/about.php)

# Augment

In [3]:
# add 'team1' and 'team2'
def augment_teams(df):
    r = pd.DataFrame()
    for gid,_df in df.groupby('GameId'):
        (t1,t2) = tuple(set(_df['OffenseTeam'].values))
        N = _df.shape[0]
        # print(f"{gid} on {_df['GameDate'].values[0]}: {t1} vs {t2}")
        _df['Team1'] = t1
        _df['Team2'] = t2
        r = pd.concat([r,_df],axis=0)
    return r

# sort games by quarter, minute, second
def sort_plays_sequentially(df):
    r = pd.DataFrame()
    for gid,_df in df.groupby('GameId'):
        _df = _df.sort_values(['Quarter','Minute','Second'],ascending=[True, False, False])
        r = pd.concat([r,_df],axis=0)
    return r

def augment(df):
    df = augment_teams(df)
    df = sort_plays_sequentially(df)
    return df
    

df = augment(df)
df.head()
    

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,Team1,Team2
4546,2024090500,2024-09-05,1,15,0,BAL,KC,0,0,65,...,NaN,35,OPP,0,NaN,0,NaN,0,KC,BAL
5197,2024090500,2024-09-05,1,15,0,BAL,KC,1,10,30,...,LEFT END,30,OWN,0,NaN,0,NaN,0,KC,BAL
5198,2024090500,2024-09-05,1,14,19,BAL,KC,2,8,32,...,NaN,32,OWN,1,BAL,1,ILLEGAL FORMATION,5,KC,BAL
5199,2024090500,2024-09-05,1,13,55,BAL,KC,2,13,27,...,NaN,27,OWN,0,NaN,0,NaN,0,KC,BAL
5200,2024090500,2024-09-05,1,13,20,BAL,KC,3,11,29,...,NaN,29,OWN,0,NaN,0,NaN,0,KC,BAL


# Get a specific game by teams

In [4]:
def get_game_by_teams(df, team1, team2):
    _ist1 = df['Team1'].isin([team1,team2])
    _ist2 = df['Team2'].isin([team1,team2])
    return df[(_ist1) & (_ist2)]['GameId'].values[0]

In [5]:
df[df['GameId']==get_game_by_teams(df, "WAS","CIN")].head()

,GameId,GameDate,Quarter,Minute,Second,OffenseTeam,DefenseTeam,Down,ToGo,YardLine,...,RushDirection,YardLineFixed,YardLineDirection,IsPenaltyAccepted,PenaltyTeam,IsNoPlay,PenaltyType,PenaltyYards,Team1,Team2
623,2024092301,2024-09-23,1,15,0,CIN,WAS,0,0,65,...,NaN,35,OPP,0,NaN,0,NaN,0,WAS,CIN
8615,2024092301,2024-09-23,1,14,54,CIN,WAS,1,10,29,...,NaN,29,OWN,0,NaN,0,NaN,0,WAS,CIN
8616,2024092301,2024-09-23,1,14,16,CIN,WAS,2,9,30,...,NaN,30,OWN,0,NaN,0,NaN,0,WAS,CIN
8617,2024092301,2024-09-23,1,13,35,CIN,WAS,3,1,38,...,NaN,38,OWN,0,NaN,0,NaN,0,WAS,CIN
8618,2024092301,2024-09-23,1,13,0,CIN,WAS,1,10,42,...,LEFT GUARD,42,OWN,0,NaN,0,NaN,0,WAS,CIN


# Possible Yards
- Remove all special teams plays
- identify each drive (first row where offensive team is different)
- remove all drives that include a kneeldown
- group by drive and compute total yards to go (factor in penalies)


| DriveId | Offensive Team | Possible Yards | Points | 
| ------- | -------------- | -------------- | ------ |
| 001     | WAS            | 73             | 7      |



In [6]:
# we only care about plays in which a ball is snapped to a non-kicker
def select_offensive_plays(df):
    return df[df['Down']!=0]

# assign a unique, serial index to each drive
def assign_drive_id(df):
    count = 0
    drive_id = []
    for i,t in enumerate(df['OffenseTeam'].values):
        if i > 0 and t!=df.iloc[i-1]['OffenseTeam']:
            count+=1
        drive_id.append(count)
    # df = df.assign(drive_id=drive_id)
    df = df.assign(drive_id=[x for x in drive_id])
    return df



# possible yards
def _compute_possible_yards(df):
    minyard = min(df['YardLine'])
    outcome = _compute_drive_outcome(df)
    if outcome=="QB KNEEL":
        return 0
    else:
        return 100 - minyard
    

# summarize outcome
def _compute_drive_outcome(df):
    if sum(df['IsTouchdown'])>0:
        return 'TOUCHDOWN'
    elif 'FIELD GOAL' in set(df['PlayType']):
        return 'FIELD GOAL'
    elif 'QB KNEEL' in set(df['PlayType']):
        return 'QB KNEEL'
    else:
        return 'FAILURE'

# compute the yards gained in the drive
def _compute_gained_yards(df):
    outcome = _compute_drive_outcome(df)
    py = _compute_possible_yards(df) # 100 - min(yl)
    if outcome=="TOUCHDOWN":
        return py
    elif outcome in {"FIELD GOAL","FAILURE"}:
        y0 = min(df['YardLine'])
        y1 = df.tail(1)['YardLine'].values[0]
        return y1-y0
    else:
        # QB KNEEL
        return 0
        

# create game summary by drive
def summarize_game_by_drive(df):
    data = []
    df = select_offensive_plays(df)
    df = assign_drive_id(df)
    gameid = df['GameId'].values[0]
    team1 = df['Team1'].values[0]
    team2 = df['Team2'].values[0]
    gamedate = df['GameDate'].values[0]
    for tid,_df in df.groupby('drive_id'):
        data.append({
            'game_id': gameid,
            'team_1': team1,
            'team_2': team2,
            'game_date': gamedate,
            'team':_df['OffenseTeam'].values[0],
            'drive':tid,
            'possible_yards': _compute_possible_yards(_df),
            'drive_outcome': _compute_drive_outcome(_df),
            'gained_yards': _compute_gained_yards(_df)
        })
    return pd.DataFrame.from_records(data)

In [7]:
def print_summary_possible_yardage(df):
    for team,_df in df.groupby('team'):
        yg = _df['gained_yards'].sum()
        yp = _df['possible_yards'].sum()
        print(f"{team} gained a total {yg} of a possible {yp} yards")

## Examples: WAS vs CIN

In [8]:
dfg = df[df['GameId']==get_game_by_teams(df, "WAS","CIN")]
dfg_summary = summarize_game_by_drive(dfg)
print(dfg_summary)
print('------------------------')
print_summary_possible_yardage(dfg_summary)

       game_id team_1 team_2   game_date team  drive  possible_yards  \
0   2024092301    WAS    CIN  2024-09-23  CIN      0              71   
1   2024092301    WAS    CIN  2024-09-23  WAS      1              70   
2   2024092301    WAS    CIN  2024-09-23  CIN      2              77   
3   2024092301    WAS    CIN  2024-09-23  WAS      3              62   
4   2024092301    WAS    CIN  2024-09-23  CIN      4              70   
5   2024092301    WAS    CIN  2024-09-23  WAS      5              75   
6   2024092301    WAS    CIN  2024-09-23  CIN      6              70   
7   2024092301    WAS    CIN  2024-09-23  WAS      7              70   
8   2024092301    WAS    CIN  2024-09-23  CIN      8              78   
9   2024092301    WAS    CIN  2024-09-23  WAS      9              70   
10  2024092301    WAS    CIN  2024-09-23  CIN     10              70   
11  2024092301    WAS    CIN  2024-09-23  WAS     11              75   
12  2024092301    WAS    CIN  2024-09-23  CIN     12            

## Examples: PHI vs NO

In [9]:
dfg = df[df['GameId']==get_game_by_teams(df, "PHI","NO")]
dfg_summary = summarize_game_by_drive(dfg)
print(dfg_summary)
print('------------------------')
print_summary_possible_yardage(dfg_summary)

       game_id team_1 team_2   game_date team  drive  possible_yards  \
0   2024092203    PHI     NO  2024-09-22   NO      0              70   
1   2024092203    PHI     NO  2024-09-22  PHI      1              79   
2   2024092203    PHI     NO  2024-09-22   NO      2              76   
3   2024092203    PHI     NO  2024-09-22  PHI      3              77   
4   2024092203    PHI     NO  2024-09-22   NO      4              81   
5   2024092203    PHI     NO  2024-09-22  PHI      5              79   
6   2024092203    PHI     NO  2024-09-22   NO      6              53   
7   2024092203    PHI     NO  2024-09-22  PHI      7              93   
8   2024092203    PHI     NO  2024-09-22   NO      8               0   
9   2024092203    PHI     NO  2024-09-22  PHI      9              77   
10  2024092203    PHI     NO  2024-09-22   NO     10              60   
11  2024092203    PHI     NO  2024-09-22  PHI     11              85   
12  2024092203    PHI     NO  2024-09-22   NO     12            

## Examples: NYG vs WAS

In [10]:
dfg = df[df['GameId']==get_game_by_teams(df, "WAS","NYG")]
dfg_summary = summarize_game_by_drive(dfg)
print(dfg_summary)
print('------------------------')
print_summary_possible_yardage(dfg_summary)

       game_id team_1 team_2   game_date team  drive  possible_yards  \
0   2024091509    NYG    WAS  2024-09-15  WAS      0              75   
1   2024091509    NYG    WAS  2024-09-15  NYG      1              53   
2   2024091509    NYG    WAS  2024-09-15  WAS      2              70   
3   2024091509    NYG    WAS  2024-09-15  NYG      3              76   
4   2024091509    NYG    WAS  2024-09-15  WAS      4              85   
5   2024091509    NYG    WAS  2024-09-15  NYG      5              69   
6   2024091509    NYG    WAS  2024-09-15  WAS      6               0   
7   2024091509    NYG    WAS  2024-09-15  NYG      7              74   
8   2024091509    NYG    WAS  2024-09-15  WAS      8              70   
9   2024091509    NYG    WAS  2024-09-15  NYG      9              66   
10  2024091509    NYG    WAS  2024-09-15  WAS     10              73   
11  2024091509    NYG    WAS  2024-09-15  NYG     11              70   
12  2024091509    NYG    WAS  2024-09-15  WAS     12            

# Output Drive Outcome Summary for all games

In [11]:
dfo = pd.DataFrame()
for gameid, dfg in df.groupby('GameId'):
    print(f"gameid = {gameid} ")
    dfg_summary = summarize_game_by_drive(dfg)
    dfo = pd.concat([dfo, dfg_summary],axis=0)

gameid = 2024090500 
gameid = 2024090600 
gameid = 2024090800 
gameid = 2024090801 
gameid = 2024090802 
gameid = 2024090803 
gameid = 2024090804 
gameid = 2024090805 
gameid = 2024090806 
gameid = 2024090807 
gameid = 2024090808 
gameid = 2024090809 
gameid = 2024090810 
gameid = 2024090811 
gameid = 2024090812 
gameid = 2024090900 
gameid = 2024091200 
gameid = 2024091500 
gameid = 2024091501 
gameid = 2024091502 
gameid = 2024091503 
gameid = 2024091504 
gameid = 2024091505 
gameid = 2024091506 
gameid = 2024091507 
gameid = 2024091508 
gameid = 2024091509 
gameid = 2024091510 
gameid = 2024091511 
gameid = 2024091512 
gameid = 2024091513 
gameid = 2024091600 
gameid = 2024091900 
gameid = 2024092200 
gameid = 2024092201 
gameid = 2024092202 
gameid = 2024092203 
gameid = 2024092204 
gameid = 2024092205 
gameid = 2024092206 
gameid = 2024092207 
gameid = 2024092208 
gameid = 2024092209 
gameid = 2024092210 
gameid = 2024092211 
gameid = 2024092212 
gameid = 2024092300 
gameid = 2024

In [12]:
dfo

,game_id,team_1,team_2,game_date,team,drive,possible_yards,drive_outcome,gained_yards
0,2024090500,KC,BAL,2024-09-05,BAL,0,73,TOUCHDOWN,73
1,2024090500,KC,BAL,2024-09-05,KC,1,67,TOUCHDOWN,67
2,2024090500,KC,BAL,2024-09-05,BAL,2,70,FAILURE,70
3,2024090500,KC,BAL,2024-09-05,KC,3,90,FAILURE,34
4,2024090500,KC,BAL,2024-09-05,BAL,4,80,FAILURE,0
...,...,...,...,...,...,...,...,...,...
9,2024092301,WAS,CIN,2024-09-23,WAS,9,70,FIELD GOAL,46
10,2024092301,WAS,CIN,2024-09-23,CIN,10,70,TOUCHDOWN,70
11,2024092301,WAS,CIN,2024-09-23,WAS,11,75,TOUCHDOWN,75
12,2024092301,WAS,CIN,2024-09-23,CIN,12,70,TOUCHDOWN,70


In [14]:
dfo.to_parquet('../datascience/data/game_drive_summary_2024.pq',engine='fastparquet',index=False)

In [ ]:
x = pd.read_parquet('